In [419]:
from my_trader import *
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib2
import time
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_driver.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_driver
#---------------------------
def render_text_contact(get_temp):
    flag = False
    trial = 0
    result = pd.DataFrame()
    while not flag:
        try:
            get_temp = get_temp.text.split("\n")
            flag = True
        except Exception as e:
            print "Get text problem"
            if trial > 3:
                result = [np.NaN,np.NaN,np.NaN,np.NaN]
                return result
            driver.refresh()
            time.sleep(10+2*trial)
            trial +=1
            #get_temp = get_temp.get_attribute('innerHTML')
            #print ("!!!! verify: " + get_temp)
            #time.sleep(step_wait)
            #flag= True
            
    header=["name","position","location","industry"]
    count_header = 0
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        if ":" in get_temp[i]:
            #flag = True
            get = get_temp[i].split(":")
            result = result.append([get])

        else:
            result = result.append([[header[count_header],get_temp[i]]])
            count_header +=1

    result = result.reindex()    
    return result

def get_progress():
    progress = pd.DataFrame(list(mongodb.db["hoover_data"].find())).company_name
    progress = list(progress)
    return progress
#---------------------------



In [384]:
progress = []

In [385]:

#------PARAMETERS---------------

step_wait = 2
page_wait = 5
collection_name = "California Minority Food Growers"
list_row_number = 3  #input the list position of the list you want to crawl, starting 0

#-------------------------------

In [ ]:
#flag initialize
rebot_flag = True #default to be True to start with the program 
trial = 0  #default to be 0
while True:     
    try:
        if trial > 3:
            rebot_flag = True
        
        if rebot_flag:  
            #reset flag
            trial = 0
            rebot_flag = False
        #===============OPEN BROWER=============================================
            driver=webdriver.Firefox()
            session_id =driver.session_id
            executor_url =driver.command_executor._url
            driver.implicitly_wait(5)

            driver.get('https://signon.onesource.com/Web/Logon/Signin.aspx')
            time.sleep(step_wait)
            driver.implicitly_wait(5)
            driver.find_element_by_id("username").clear()
            driver.find_element_by_id("username").send_keys("hli@calasiancc.org")
            driver.find_element_by_id("password").clear()
            driver.find_element_by_id("password").send_keys("dbhoovers1")
            driver.find_element_by_id("btnSignIn").click()
            time.sleep(step_wait)
            driver.implicitly_wait(10)
            driver.find_element_by_link_text("Lists").click()
            time.sleep(step_wait)
            driver.implicitly_wait(10)

            # list selector

            list_ele = driver.find_elements_by_class_name("name-row")
            get = list_ele[list_row_number]
            get.find_element_by_tag_name("a").click()



            #page info

            current_page = driver.find_element_by_class_name("current-page")
            current_page=current_page.get_attribute('innerHTML')
            current_page = int(current_page)

            total_page = driver.find_element_by_class_name("total-page")
            total_page=total_page.get_attribute('innerHTML')
            total_page = int(total_page)
    #================================================================================
            
        driver.refresh()
        driver.implicitly_wait(100)
        current_page = driver.find_element_by_class_name("current-page")
        current_page=current_page.get_attribute('innerHTML')
        current_page = int(current_page)

    # company selector
        company_list = driver.find_elements_by_class_name("clickable")
        name_list=[]
        for i in range(len(company_list)):
            name_list.append(company_list[i].text)    

        for i in name_list:
            #clear_output()

            #company_list = driver.find_elements_by_class_name("clickable")        
            #get_company = company_list[i]
            if i in progress:
                print (i+ " already in database, skipping")
                continue
            time.sleep(page_wait)
            driver.implicitly_wait(10)   
            get_company = driver.find_element_by_link_text(i)
            driver.execute_script("return arguments[0].scrollIntoView();", get_company)
            driver.execute_script("arguments[0].click();", get_company)
            #company_list[i].click()
            WebDriverWait(driver, 80).until(EC.presence_of_element_located((By.ID, "totalContacts")))
            time.sleep(step_wait)
            #get basic information
            info = {}
            address = driver.find_element_by_class_name("company-address").text
            company_name = driver.find_element_by_class_name("name").text
            try:
                website = driver.find_element_by_class_name("url").text
            except:
                print ("no website found")
            data = driver.find_elements_by_class_name("data-container")
            for i in range(len(data)):

                try:
                    label=data[i].find_element_by_class_name("data-label")
                except:
                    continue 
                    #label=data[i].find_element_by_class_name("data-label wide")
                value=data[i].find_elements_by_class_name("data-value")
                value = value[0].text
                label = label.text
                label = label.encode('ascii','ignore')
                info[label] = value

            info["address"] = address
            info["company_name"] = company_name
            info["website"] = website
            mongodb.db[collection_name].insert_one(info)
            print (company_name + ": basic information got")
            #get description about the company

            data_title = ["description","industry","highlights"]
            description=pd.DataFrame()
            for i in data_title:
                get_temp = driver.find_element_by_class_name("highlights")
                get_temp = get_temp.text.split("\n")
                #get_temp = get_temp.text.split(":")
            for i in range(len(get_temp)):
                get_temp[i] = get_temp[i].encode('ascii','ignore')
                get = get_temp[i].split(":")
                description = description.append([get])
            description = description.set_index(0)
            description=json.loads(description.to_json()).values()[0]
            mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":description},upsert=True)
            print (company_name + ": description information got")

            contacts_flag = False

            while not contacts_flag:
                try:
                    contacts = driver.find_element_by_id("totalContacts").text
                    contacts = int(contacts)
                    contacts_flag = True
                except:
                    #print ("contact number didn't come out yet")
                    continue
            if contacts != 0:
                try:
                    driver.find_element_by_id("company_contacts").click()
                except:
                    driver.execute_script("arguments[0].click();", driver.find_element_by_id("company_contacts"))
                time.sleep(step_wait)
                WebDriverWait(driver, step_wait).until(EC.presence_of_element_located((By.CLASS_NAME, "detail-container")))                                   
                contacts = driver.find_elements_by_class_name("detail-container")

                contacts = driver.find_elements_by_class_name("search-result-container")
                result = []
                for i in range(len(contacts)):

                    temp = render_text_contact(contacts[i])
                    temp = temp.set_index(temp[0])
                    temp = temp.drop(0,axis=1)
                    result.append(json.loads(temp.to_json()).values())
                mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":{"contact":result}},upsert=True)
                print (company_name + ": contacts information got")
                driver.back()
            else:
                print ("no contact available")

            # page wait time   
            progress.append(company_name)
            driver.back()

        # next page
        if total_page > current_page:
            next_page = driver.find_element_by_class_name("next-page")
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(page_wait)
            driver.implicitly_wait(10) 
        else:
            break
    except Exception as e:
        print ("---------------")
        print e
        print ("---------------")
        trial += 1
        continue
#================================================    
#Getting hoover data

hoover_data = pd.DataFrame(list(mongodb.db[collection_name].find())).drop("",axis = 1)
hoover_data=hoover_data.set_index("company_name")
hoover_data=hoover_data.loc[:,"contact"].dropna()
#Getting hoover contact data
hoover_contacts = pd.DataFrame()
for i in range(len(hoover_data)):
    for j in hoover_data.iloc[i]:
        temp = pd.DataFrame(j,index=[hoover_data.index[i]])
        hoover_contacts = hoover_contacts.append(temp)

hoover_contacts.index.name = "company_name"
hoover_data.to_csv(collection_name + ".csv")
hoover_contacts.to_csv(collection_name+"_contacts.csv")

Ramco Enterprises, L.P. already in database, skipping
Vallarta Supermarkets, Inc. already in database, skipping
Ruiz Food Products, Inc. already in database, skipping
Sweet Candy, LLC already in database, skipping
MI Pueblo Newco, LLC already in database, skipping
Tawa Supermarket, Inc. already in database, skipping
Teasdale Foods, Inc. already in database, skipping
Red Chamber Co. already in database, skipping
Fuji Food Products, Inc. already in database, skipping
Uka's Big Saver Foods, Inc. already in database, skipping
Marquez Brothers International, Inc. already in database, skipping
Asiana Cuisine Enterprises, Inc. already in database, skipping
Harris Freeman & Co., Inc. already in database, skipping
San Miguel Produce, Inc. already in database, skipping
Ramco Enterprises, L.P. already in database, skipping
Camino Real Foods, Inc. already in database, skipping
El Tigre, Inc. already in database, skipping
Valley Fine Foods Company, Inc. already in database, skipping
Esparza Enterpr

Saab Enterprises Inc already in database, skipping
Sogo Bakery, Inc. already in database, skipping
Heartland Meat Company, Inc. already in database, skipping
Sun Hing Foods, Inc. already in database, skipping
Unified Food Ingredients, Inc already in database, skipping
Cal Fresco, LLC already in database, skipping
Sun Lee, Inc. already in database, skipping
Amay's Bakery & Noodle Co., Inc. already in database, skipping
Reliance Intermodal Inc. already in database, skipping
Risvold's, Inc. already in database, skipping
Uka's Big Saver Foods, Inc. already in database, skipping
Vegiworks, Inc. already in database, skipping
Bazan Mario AG Services & Vine already in database, skipping
La Mousse already in database, skipping
Frank's Markets, A California Corporation already in database, skipping
W B Wright, Inc already in database, skipping
Maruhide Marine Products Inc already in database, skipping
Boghosian Raisin Packing Company, Inc. already in database, skipping
Willow Farms LLC already i

Pin Hsiao & Associates L.L.C. already in database, skipping
Acapulco Mexicatessen Inc already in database, skipping
Farmers Market already in database, skipping
3 Little Cinnamen Inc already in database, skipping
Formosa Meat Company Inc already in database, skipping
Phoenix Bakery Inc already in database, skipping
Arko Foods International, Inc. already in database, skipping
Foodnet Supermarket Inc already in database, skipping
Rama Food Manufacture Corp. already in database, skipping
Sumano's Bakery, LLC already in database, skipping
Elite Business Ventures, Inc already in database, skipping
P & D Seafood Company already in database, skipping
Mercado Latino, Inc. already in database, skipping
Organic By Nature, Inc. already in database, skipping
Chun & Lee Inc already in database, skipping
Uka's Big Saver Foods, Inc. already in database, skipping
Ub, Inc. already in database, skipping
Uka's Big Saver Foods, Inc. already in database, skipping
Ikeda Bros already in database, skipping
Ki

Minh's Meat Co., Inc. already in database, skipping
Major Cash & Carry Inc already in database, skipping
Fagundes Agribusiness already in database, skipping
Gourmet Express already in database, skipping
Suruki Super Market Inc already in database, skipping
Ito Bros., Inc. already in database, skipping
Freska Produce International, LLC already in database, skipping
Kims Farms Inc already in database, skipping
Lakeview Ranch LP already in database, skipping
Socially Conscious and Green already in database, skipping
Harvest Foods Inc already in database, skipping
Raquel's Cash N' Carry, LLC already in database, skipping
Island Snacks, Inc. already in database, skipping
L A Cathay Inc already in database, skipping
Fung's Village, Inc. already in database, skipping
Pay-Low Super Market already in database, skipping
Saroma, Inc. already in database, skipping
Flor De California already in database, skipping
Pizzico, Inc. already in database, skipping
West Bay Seafood Company, Inc. already in 

Sunnylife Global, Inc. already in database, skipping
R & L Rodarte Inc already in database, skipping
Dtbm Inc already in database, skipping
El Quetzal Bakery already in database, skipping
Hydrosprout Inc already in database, skipping
Stockton Gateway Fueling Inc already in database, skipping
Dialco Restaurants, Inc. already in database, skipping
G & P Group, Inc already in database, skipping
Chino Meats Provisions Corp already in database, skipping
Carniceria El Matador already in database, skipping
Kuramura Nursery Inc already in database, skipping
Valley Fine Foods Company, Inc. already in database, skipping
Rusemad Ventures, Inc. already in database, skipping
Rancho Grande Meat Market already in database, skipping
Robeks Juice La06 already in database, skipping
H E W Drilling Company Inc already in database, skipping
Mayfield Equipment Company already in database, skipping
Minami Greenhouse Inc already in database, skipping
Lap Tang International already in database, skipping
M Tash

...

EFC Corp. already in database, skipping
Shirley's Bagels Etc already in database, skipping
Everest Grocers & Vending Inc already in database, skipping
David Hong already in database, skipping
A & F Growers, Inc already in database, skipping
Mac's Grog & Grocery already in database, skipping
Angel Doughnuts & Cakes already in database, skipping
Key Market Liquor already in database, skipping
Farmers Empire, LLC already in database, skipping
Mrs. Seafood Inc. already in database, skipping
New India Sweets & Spices already in database, skipping
Morelia Food Products already in database, skipping
Fair Trade Market already in database, skipping
Golden State Provisions Inc. already in database, skipping
Sunny Donuts already in database, skipping
Sandy Bay already in database, skipping
Rainbow Donuts already in database, skipping
Apple-A-Day Health Food Co already in database, skipping
Garcia Market already in database, skipping
Lupita Panaderia already in database, skipping
H2rose LLC alread

Agri-Pacific International Trading Corporation already in database, skipping
K & T Donuts already in database, skipping
Nancy's Market and Restaurant already in database, skipping
Gene's Market & Liquor already in database, skipping
C & R Farms already in database, skipping
E & M Corp. already in database, skipping
Rosa's Bakery, Inc already in database, skipping
Atlas Health & Nutrition Inc already in database, skipping
Calmonte Corporation already in database, skipping
Nature Friend Inc already in database, skipping
El Rancho Market already in database, skipping
Seven Liquors already in database, skipping
Ten Yo Enterprises Inc already in database, skipping
Lucero Express Bakery already in database, skipping
Heavenly Cakes Creations already in database, skipping
Supreme Donuts already in database, skipping
Mary Shihadeh already in database, skipping
Sweet Candy, LLC already in database, skipping
V P Discount Corporation already in database, skipping
Gonzalo R Soto already in database

Gonzalez Farms already in database, skipping
Puni Enterprises Inc already in database, skipping
Dolly's Doughnut already in database, skipping
Alpine Frontier Liquor already in database, skipping
Jalo's Market already in database, skipping
La Fortuna Market already in database, skipping
El Papucho Mexican Imports already in database, skipping
Jax Doughnuts 15 already in database, skipping
Espinoza Meat Market already in database, skipping
H Food Mart already in database, skipping
Lisa's Market already in database, skipping
Mini Tropicana already in database, skipping
Fairway Market already in database, skipping
Lunch Bunch Deli already in database, skipping
Marquez Brothers Texas, LP already in database, skipping
Richland Enterprises LLC already in database, skipping
El Rey Supermarket already in database, skipping
Asia Mart already in database, skipping
K and H Market already in database, skipping
So Alive Company, LLC already in database, skipping
Jello-King Corp already in database,

Driftwood Dairy already in database, skipping
Valerios' City Bakery already in database, skipping
Colenso Inc. already in database, skipping
Farm Stand Fresh Foods, Inc. already in database, skipping
Aldama's Carniceria Y Market already in database, skipping
Krispys Donuts already in database, skipping
Phoenix Market already in database, skipping
H & P Market already in database, skipping
Fryers Custom Meat Cutting already in database, skipping
Maras Italian Pastry already in database, skipping
Sundance Market already in database, skipping
Wang Chung Company already in database, skipping
Pomona Fish Market already in database, skipping
Jewel Meat Inc already in database, skipping
Regional Del Valle already in database, skipping
Kemi Enterprises Inc already in database, skipping
828 Coffee, Inc. already in database, skipping
Chaquito's Bakery already in database, skipping
Las Victorias Bakery already in database, skipping
Keynex Trading Corporation already in database, skipping
Quick Pi

Anita's Mexican Foods Corp. already in database, skipping
Ramona's Mexican Food Products, Inc. already in database, skipping
MI Pueblo Newco, LLC already in database, skipping
Delta Breeze Farming, Inc. already in database, skipping
Marquez Brothers Enterprises, Inc. already in database, skipping
C & G Farms, Inc. already in database, skipping
Berkeley Bowl Produce, Inc. already in database, skipping
Ultimate Formulations Inc. already in database, skipping
Mother's Nutritional Center, Inc. already in database, skipping
WEI-Chuan U.S.A., Inc. already in database, skipping
Tawa Supermarket, Inc. already in database, skipping
Coast Produce Company already in database, skipping
Tawa Supermarket, Inc. already in database, skipping
Rizo-Lopez Foods, Inc. already in database, skipping
J.R. Simplot Company already in database, skipping
La Tapatia Tortilleria, Inc. already in database, skipping
H & N Foods International Inc already in database, skipping
Le Pafe, Inc. already in database, skippi